# Diabetic Data Cleaning Notebook
This notebook performs Phase 1 of the data cleaning process for the diabetic dataset. It loads the raw data, handles missing values, removes expired patients, merges ID descriptions, and validates the final cleaned dataset.

In [1]:
import pandas as pd
import numpy as np
import os

## 1️⃣ Load the raw diabetic data

In [2]:
# File paths (adjust if needed)
RAW_DATA_PATH = '../data/raw/diabetic_data.csv'
ID_MAP_PATH = '../data/raw/IDs_mapping.csv'
OUTPUT_PATH = '../data/processed/diabetic_data_clean.csv'

# Load raw data, converting '?' to NaN
df = pd.read_csv(RAW_DATA_PATH, na_values=['?'])
df.head()

C:\Users\kames\AppData\Local\Temp\ipykernel_10792\287847556.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_DATA_PATH, na_values=['?'])


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


## 2️⃣ Drop `weight` column due to excessive missing values

In [3]:
if df['weight'].isna().mean() > 0.9:
    df = df.drop(columns=['weight'])
    print('Dropped weight column due to >90% missing values')

Dropped weight column due to >90% missing values


## 3️⃣ Remove expired patients

In [4]:
expired_ids = [11,19,20,21]  # Expired discharge_disposition_id
df = df[~df['discharge_disposition_id'].isin(expired_ids)]
print(f'Remaining rows after removing expired patients: {len(df)}')

Remaining rows after removing expired patients: 100114


## 4️⃣ Load and merge ID mappings

In [5]:
# ---------------- 4️⃣ Load and merge ID mappings ---------------- #

# Function to parse the IDs_mapping.csv into separate DataFrames
def load_mapping_sections(mapping_csv_path):
    """
    Parse IDs_mapping.csv which contains multiple mapping tables
    separated by header rows and blank lines.
    Returns three DataFrames: admission_type, discharge_disposition, admission_source
    """
    sections = {
        "admission_type_id": [],
        "discharge_disposition_id": [],
        "admission_source_id": []
    }

    current_section = None

    with open(mapping_csv_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            # Detect section headers
            if line.startswith("admission_type_id"):
                current_section = "admission_type_id"
                continue
            elif line.startswith("discharge_disposition_id"):
                current_section = "discharge_disposition_id"
                continue
            elif line.startswith("admission_source_id"):
                current_section = "admission_source_id"
                continue

            # Skip malformed lines
            if "," not in line or current_section is None:
                continue

            id_val, desc = line.split(",", 1)
            if id_val.isdigit():
                sections[current_section].append(
                    {"id": int(id_val), "description": desc.strip('"')}
                )

    # Convert each section to DataFrame
    admission_type_df = pd.DataFrame(sections["admission_type_id"])
    discharge_df = pd.DataFrame(sections["discharge_disposition_id"])
    admission_source_df = pd.DataFrame(sections["admission_source_id"])

    return admission_type_df, discharge_df, admission_source_df



In [6]:
# Load mapping CSV
admission_type_map, discharge_map, admission_source_map = load_mapping_sections(ID_MAP_PATH)


### Note: The mapping CSV contains multiple sections. For simplicity, we'll create separate mappings manually based on the earlier CSV content.

In [7]:
# Function to map numeric IDs to descriptions
def merge_id_descriptions_safe(df, mapping_df, id_col, new_col_name):
    """
    Maps numeric IDs to descriptions safely without creating extra columns.
    """
    mapping_dict = mapping_df.set_index('id')['description'].to_dict()
    df[new_col_name] = df[id_col].map(mapping_dict)
    print(f"Mapped '{id_col}' to '{new_col_name}'")
    return df

# Merge all three mappings into the main DataFrame
df = merge_id_descriptions_safe(df, admission_type_map, 'admission_type_id', 'admission_type_desc')
df = merge_id_descriptions_safe(df, discharge_map, 'discharge_disposition_id', 'discharge_desc')
df = merge_id_descriptions_safe(df, admission_source_map, 'admission_source_id', 'admission_source_desc')

# Check results
df.head()


Mapped 'admission_type_id' to 'admission_type_desc'
Mapped 'discharge_disposition_id' to 'discharge_desc'
Mapped 'admission_source_id' to 'admission_source_desc'


,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admission_type_desc,discharge_desc,admission_source_desc
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,NaN,...,No,No,No,No,No,No,NO,NULL,Not Mapped,Physician Referral
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,NaN,...,No,No,No,No,Ch,Yes,>30,Emergency,Discharged to home,Emergency Room
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,NaN,...,No,No,No,No,No,Yes,NO,Emergency,Discharged to home,Emergency Room
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,NaN,...,No,No,No,No,Ch,Yes,NO,Emergency,Discharged to home,Emergency Room
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,NaN,...,No,No,No,No,Ch,Yes,NO,Emergency,Discharged to home,Emergency Room


## 5️⃣ Remove duplicate rows

In [8]:
initial_count = len(df)
df = df.drop_duplicates()
print(f'Removed {initial_count - len(df)} duplicate rows.')

Removed 0 duplicate rows.


## 6️⃣ Validate cleaned dataset

In [9]:
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
Index: 100114 entries, 0 to 101765
Data columns (total 52 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              100114 non-null  int64 
 1   patient_nbr               100114 non-null  int64 
 2   race                      97875 non-null   object
 3   gender                    100114 non-null  object
 4   age                       100114 non-null  object
 5   admission_type_id         100114 non-null  int64 
 6   discharge_disposition_id  100114 non-null  int64 
 7   admission_source_id       100114 non-null  int64 
 8   time_in_hospital          100114 non-null  int64 
 9   payer_code                60523 non-null   object
 10  medical_specialty         50985 non-null   object
 11  num_lab_procedures        100114 non-null  int64 
 12  num_procedures            100114 non-null  int64 
 13  num_medications           100114 non-null  int64 
 14  number_ou

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,...,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,admission_type_desc,discharge_desc,admission_source_desc
count,1.001140e+05,1.001140e+05,97875,100114,100114,100114.000000,100114.000000,100114.000000,100114.000000,60523,...,100114,100114,100114,100114,100114,100114,100114,100114,100114,100114
unique,NaN,NaN,5,3,10,NaN,NaN,NaN,NaN,17,...,2,2,2,2,2,2,3,8,23,17
top,NaN,NaN,Caucasian,Female,[70-80),NaN,NaN,NaN,NaN,MC,...,No,No,No,No,No,Yes,NO,Emergency,Discharged to home,Emergency Room
freq,NaN,NaN,74845,53861,25562,NaN,NaN,NaN,NaN,31739,...,100101,100113,100112,100113,53671,77272,53212,52884,60234,56363
mean,1.653032e+08,5.430611e+07,NaN,NaN,NaN,2.028597,3.594622,5.738388,4.389646,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,1.027194e+08,3.871493e+07,NaN,NaN,NaN,1.446225,5.237490,4.066412,2.974531,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.252200e+04,1.350000e+02,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,8.493462e+07,2.339867e+07,NaN,NaN,NaN,1.000000,1.000000,1.000000,2.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.524511e+08,4.548078e+07,NaN,NaN,NaN,1.000000,1.000000,7.000000,4.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.305332e+08,8.755836e+07,NaN,NaN,NaN,3.000000,3.000000,7.000000,6.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 7️⃣ Save cleaned dataset

In [10]:
df.to_csv(OUTPUT_PATH, index=False)
print(f'Saved cleaned dataset to {OUTPUT_PATH}')

Saved cleaned dataset to ../data/processed/diabetic_data_clean.csv


## Cleaning Decisions Summary:
- **Missing values**: Replaced '?' with NaN for proper handling.
- **Weight column**: Dropped due to >90% missing values.
- **Expired patients**: Removed patients with `discharge_disposition_id` 11.
- **ID mappings**: Merged human-readable descriptions for admission type, discharge disposition, and admission source.
- **Duplicates**: Removed duplicate rows to ensure clean data.
- **Validation**: Dataset info and descriptive stats checked post-cleaning.